In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dense, ZeroPadding2D, Activation
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import warnings
from os import *
warnings.filterwarnings('ignore')

In [2]:
x = []
y = []
for directory in ['augmented data/yes', 'augmented data/no']:
	for filename in listdir(directory):
		image = cv2.imread(directory + '/' + filename)
		image = cv2.resize(image, dsize=(240,240), interpolation=cv2.INTER_CUBIC)
		image = image / 255.
		x.append(image)
		if directory[-3:] == 'yes':
			y.append([1])
		else:
			y.append([0])

In [3]:
x = np.array(x)
y = np.array(y)
x, y = shuffle(x, y)

print(f'Number of samples : {len(x)}')
print(f'x shape : {x.shape}')
print(f'y shape : {y.shape}')

Number of samples : 2065
x shape : (2065, 240, 240, 3)
y shape : (2065, 1)


In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=101)

In [5]:
input = Input((240,240,3))
model = ZeroPadding2D((2, 2))(input)
model = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(model)
model = BatchNormalization(axis = 3, name = 'bn0')(model)
model = Activation('relu')(model)
model = MaxPooling2D((4, 4), name='max_pool0')(model) 
model = MaxPooling2D((4, 4), name='max_pool1')(model)
model = Flatten()(model)
model = Dense(1, activation='sigmoid', name='fc')(model)
model = Model(inputs = input, outputs = model, name='BrainTumorDetection')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
model.summary()

Model: "BrainTumorDetection"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 zero_padding2d (ZeroPadding  (None, 244, 244, 3)      0         
 2D)                                                             
                                                                 
 conv0 (Conv2D)              (None, 238, 238, 32)      4736      
                                                                 
 bn0 (BatchNormalization)    (None, 238, 238, 32)      128       
                                                                 
 activation (Activation)     (None, 238, 238, 32)      0         
                                                                 
 max_pool0 (MaxPooling2D)    (None, 59, 59, 32)        0         
                                               

In [7]:
model.fit(x=xtrain, y=ytrain, batch_size=32, epochs=10)

Epoch 1/10
52/52 [==============================] - 99s 2s/step - loss: 0.6859 - accuracy: 0.7040
Epoch 2/10
52/52 [==============================] - 92s 2s/step - loss: 0.4865 - accuracy: 0.7766
Epoch 3/10
52/52 [==============================] - 101s 2s/step - loss: 0.4639 - accuracy: 0.7887
Epoch 4/10
52/52 [==============================] - 105s 2s/step - loss: 0.4682 - accuracy: 0.7827
Epoch 5/10
52/52 [==============================] - 118s 2s/step - loss: 0.3870 - accuracy: 0.8202
Epoch 6/10
52/52 [==============================] - 107s 2s/step - loss: 0.3951 - accuracy: 0.8317
Epoch 7/10
52/52 [==============================] - 103s 2s/step - loss: 0.3603 - accuracy: 0.8420
Epoch 8/10
52/52 [==============================] - 103s 2s/step - loss: 0.3213 - accuracy: 0.8638
Epoch 9/10
52/52 [==============================] - 105s 2s/step - loss: 0.3787 - accuracy: 0.8305
Epoch 10/10
52/52 [==============================] - 105s 2s/step - loss: 0.3289 - accuracy: 0.8523


In [8]:
test_output = np.round(model.predict(xtest))
train_output = np.round(model.predict(xtrain))

In [9]:
print(confusion_matrix(ytest, test_output))
print(confusion_matrix(ytrain, train_output))

[[168  22]
 [ 87 136]]
[[765  25]
 [244 618]]


In [10]:
model.save('BrainTumorModel.h5')